In [1]:
'''手动读取.h5文件赋值给torch'''
import keras
from models.DFCAN16 import DFCAN as keras_DFCAN
from model.DFCAN import DFCAN as torch_DFCAN
input_height, input_width, n_channel = 502, 502, 1

weight_file = "model_weights.h5"

keras_model = keras_DFCAN((input_height, input_width, n_channel))
keras_model.load_weights(weight_file)

Using TensorFlow backend.
/root/miniconda3/envs/k2t_new/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#加载keras中可以学习的层的权重
keras_layers_name = []
keras_layers_weight = []
for layer in keras_model.layers:
    if layer.trainable == True:
        keras_layers_name.append(layer.name)
        keras_layers_weight.append(layer.weights)

In [3]:
#加载pytorch权重
torch_layers_names = []
torch_layers_params = []
model =  torch_DFCAN((n_channel))
for name, param in model.named_parameters():
    torch_layers_names.append(name)
    if param.requires_grad:
        # print(name, param.data)
        torch_layers_params.append(param)

In [4]:
import torch

In [5]:
#赋值所有层的权重
for indx, weight in enumerate(keras_model.get_weights()):
    # weight = keras_model.get_weights()[0]   #keras第一层权重：numpy.ndarray
    if len(weight.shape) == 4:  #卷积核
        to_torch_weight = torch.Tensor(weight).permute(3,2,0,1) 
    elif len(weight.shape) == 1:  #偏置
        to_torch_weight = torch.Tensor(weight)
    else:
        raise BaseException
    # to_torch_weight = torch.Tensor(weight).permute(3,2,1,0)   
    [k for k in model.named_parameters()][indx][1].data = to_torch_weight
    # print(to_torch_weight[0,0,:,:])

In [6]:
#生成固定输入
torch.manual_seed(123)
x0 = torch.randn(1, 1, 502, 502)

In [7]:
#定义用于pytorch打印的hook函数
torch_inter_layer_out = None
def get_layer_output(layer, input, output):
    global torch_inter_layer_out
    torch_inter_layer_out = output

In [8]:
def get_k_layer_name(indx):
    #获取keras模型第k层的layer_name
    #获取keras_model所有层。实际上就是83个卷积层
    trainable_layer_names = [layer.name.split(r"/")[0] for layer in keras_model.trainable_weights]
    from collections import OrderedDict
    ordered_dict = OrderedDict.fromkeys(trainable_layer_names)
    set_trainable_layer_names = list(ordered_dict.keys())
    # set_trainable_layer_names
    # keras_model.get_layer(set_trainable_layer_names[0])
    return set_trainable_layer_names[indx]

In [9]:
#获取torch model要插入hook的hook_layer
pre_index = 2

import re
s = torch_layers_names[pre_index*2]
s = re.sub(r'\.(\d+)\.', r'[\1].', s)
s = re.sub(r'\.(\d+)$', r'[\1]', s)
s = re.sub(r'\.[^.]*$', '', s)
print(s)

def get_member(cls, string):
    # 将输入字符串按照`.`分割成列表
    attribute_list = string.split(".")
    
    # 定义变量result，初始值为A
    result = cls
    
    # 遍历列表，访问对应的成员
    for attribute_name in attribute_list:
        if "[" in attribute_name and "]" in attribute_name:
            # 如果元素是以[数字]结尾的形式，就将数字提取出来，并使用getattr()函数访问result的对应成员
            index = int(attribute_name[attribute_name.index("[")+1:attribute_name.index("]")])
            attribute_name = attribute_name[:attribute_name.index("[")]
            result = getattr(result, attribute_name)[index]
        else:
            # 否则，直接使用getattr()函数访问result的对应成员
            result = getattr(result, attribute_name)
    return result

RGs[0].RCABs[0].conv_gelu2[0]


In [10]:
# 检查第layer_index层
layer_index = pre_index   
hook_layer = get_member(model,s)
# hook_layer = model.RGs[0].RCABs[0].conv_gelu1[0]
#pytorch
hook_layer.register_forward_hook(get_layer_output) #添加钩子
model(x0)
torch_out = torch_inter_layer_out

#keras
from keras import backend as K
# keras_x0 = K.constant(x0.permute(0,2,3,1).numpy())  #转到keras的输入
keras_x0 = x0.permute(0,2,3,1).numpy()
# print(type(keras_x0))
# print(keras_x0.shape)
from keras.models import Model
layer_name = get_k_layer_name(layer_index)  #keras第
intermediate_layer_model = Model(inputs=keras_model.input, outputs=keras_model.get_layer(layer_name).output)
keras_output = intermediate_layer_model.predict(keras_x0)
# print("keras卷积层输出：",type(keras_output))
keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

#比较二者
# assert torch_out.shape == keras_output_torch_tensor.shape
# print(torch_out == keras_output_torch_tensor)
print("torch模型和keras模型此层输出shape：",torch_out.shape, keras_output_torch_tensor.shape)
print("torch模型和keras模型此层输出截取：",torch_out[0,0,:3,:3], keras_output_torch_tensor[0,0,:3,:3],sep="\n")

torch模型和keras模型此层输出shape： torch.Size([1, 64, 502, 502]) torch.Size([1, 64, 502, 502])
torch模型和keras模型此层输出截取：
tensor([[ 0.0185, -0.0489,  0.0495],
        [ 0.0007, -0.0276,  0.0331],
        [-0.0091,  0.1517, -0.1249]], grad_fn=<SliceBackward0>)
tensor([[1.1116, 0.7875, 0.9944],
        [1.2446, 0.7724, 0.9506],
        [1.2026, 0.5845, 0.9090]])
